In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *
import matplotlib as mpl
import matplotlib.pyplot as plt

np.set_printoptions(precision=4)
import math
from utils import *
import itertools



In [2]:
# Test with tau=0.9
size = 8
nA = 2
seed = 3231
status_step = 500
gamma = 0.9
river = River(size, gamma, 5, 1000)
tau = 0.3
xi = np.ones(river.size)*1/river.size
tmdp = TMDP(river, xi, tau, gamma, seed)
tmdp_0 = TMDP(river, xi, 0, gamma, seed)
tmdp_1 = TMDP(river, xi, 0.02, gamma, seed)

colors = itertools.cycle(["r", "b"])

In [3]:
# Optimal Q function
r_s_a = compute_r_s_a(tmdp.size, tmdp.nA, tmdp.P_mat, tmdp.reward)
Q_star = bellman_optimal_q(tmdp.size, tmdp.nA, tmdp.P_mat, tmdp.reward, 1e-4, gamma)
J_star = get_expected_avg_reward(tmdp.P_mat, get_policy(Q_star), tmdp.reward, tmdp.gamma, tmdp.mu)
delta_q = np.linalg.norm(Q_star - Q_star, np.inf)

## 1-step transfer learning test. From tau to original problem

In [4]:
taus = [0.2, 0.02]
results = []
intermediate_results = {}
curr_j = []
delta_qs = []
status_step = 200
tmdp_0 = TMDP(river, xi, 0, gamma)
models = {}
# Learning from scratch with different values of tau
for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    Q = np.zeros((tmdp.size, tmdp.nA))
    M = 300000
    
    s = tmdp.reset()
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    
    Q, inner_main_J, inner_curr_J, inner_delta_qs, inner_delta_J, inner_l_bounds = Q_learning(tmdp, s, a, Q, Q_star, M=M, alpha=0.03, debug=True, status_step=status_step, main_p=False)
    
    intermediate_results['tau_'+str(tau)] = {'J_main_p': inner_main_J,"J_curr_p":inner_curr_J, "delta_q":inner_delta_qs, 'Q':Q, "tmdp":tmdp, "delta_J":inner_delta_J, "l_bounds": inner_l_bounds}
    delta_q = np.linalg.norm(Q_star - Q, np.inf)
    
    J = get_expected_avg_reward(tmdp.P_mat, get_policy(Q), tmdp.reward, tmdp.gamma, tmdp.mu)
    results.append({"type":"scratch", "Q": Q, "tau":"{:.2f}".format(tau), "J":J, "policy":get_policy(Q),"delta_q":delta_q, "tmdp":tmdp})
    models[str(tau)] = {'Q':Q, 'tmdp':tmdp}
"""
    s = tmdp_0.reset()
    a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])
    Q, inner_main_J, inner_curr_J, inner_delta_qs, _, _ = Q_learning(tmdp_0, s, a, Q, Q_star, M=M, alpha=0.03, debug=True, status_step=status_step)
    intermediate_results['transfer_'+str(tau)] = {'J_main_p': inner_main_J,"J_curr_p":inner_curr_J, "delta_q":inner_delta_qs, 'Q':Q, "tmdp":tmdp_0}
    delta_q = np.linalg.norm(Q_star - Q, np.inf)

    J = get_expected_avg_reward(tmdp_0.P_mat, get_policy(Q), tmdp_0.reward, tmdp_0.gamma, tmdp_0.mu)
    results.append({"type":"transfer", "Q": Q, "tau":"{:.2f}".format(tau), "J":J, "policy":get_policy(Q),"delta_q":delta_q, "tmdp":tmdp_0})"""

'\n    s = tmdp_0.reset()\n    a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])\n    Q, inner_main_J, inner_curr_J, inner_delta_qs, _, _ = Q_learning(tmdp_0, s, a, Q, Q_star, M=M, alpha=0.03, debug=True, status_step=status_step)\n    intermediate_results[\'transfer_\'+str(tau)] = {\'J_main_p\': inner_main_J,"J_curr_p":inner_curr_J, "delta_q":inner_delta_qs, \'Q\':Q, "tmdp":tmdp_0}\n    delta_q = np.linalg.norm(Q_star - Q, np.inf)\n\n    J = get_expected_avg_reward(tmdp_0.P_mat, get_policy(Q), tmdp_0.reward, tmdp_0.gamma, tmdp_0.mu)\n    results.append({"type":"transfer", "Q": Q, "tau":"{:.2f}".format(tau), "J":J, "policy":get_policy(Q),"delta_q":delta_q, "tmdp":tmdp_0})'

In [5]:
agg_res = aggregate_results(results, len(results))

In [6]:
x = np.array([])
y = np.array([])
ci = np.array([])
for res in agg_res:
    yi =res['avg_J']
    y = np.append(y, yi)
    label = res['type'] + " " + str(res['tau'])
    x = np.append(x, label)
    ci = np.append(ci, res['ci_J'])
    plt.scatter(label, yi, c=next(colors))
x_map = np.array([i for i in range(len(x))])
offset = np.array([0.25 if ci[i] > 0.001 else 0 for i in range(len(x))])
#fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')

plt.xticks(x_map, x)
#plt.plot(x, y)
#plt.fill_between(x, (y-ci), (y+ci), color='b', alpha=.1)
plt.hlines(y=y-ci, xmin=x_map-offset, xmax=x_map + offset)
plt.hlines(y=y+ci, xmin=x_map-offset, xmax=x_map + offset)
plt.hlines(y=J_star, xmin=0, xmax=len(x)-1, colors=['green'], label='Optimal J')
plt.vlines(x, ymin = y-ci, ymax = y+ci)
plt.xlabel("Q functions")
plt.xticks(rotation=90)
plt.ylabel("E[J(τ)]")
plt.title("Expected discounted sum of rewards")
plt.grid(visible=True)
plt.legend(loc="lower right")

In [7]:
x = np.array([])
y = np.array([])
ci = np.array([])
for res in agg_res:
    yi = res['avg_delta_q']
    y = np.append(y, yi)
    label = res['type'] + " " + str(res['tau']) 
    x = np.append(x, label)
    ci = np.append(ci, res['ci_delta_q'])
    plt.scatter(label, yi, c=next(colors))
offset = np.array([0.25 if ci[i] > 0.001 else 0 for i in range(len(x))])
#fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')
#plt.scatter(x, y, c='orange')

plt.xticks(x_map, x)
#plt.plot(x, y)
#plt.fill_between(x, (y-ci), (y+ci), color='b', alpha=.1)
plt.hlines(y=y-ci, xmin=x_map-offset, xmax=x_map + offset)
plt.hlines(y=y+ci, xmin=x_map-offset, xmax=x_map + offset)
plt.vlines(x, ymin = y-ci, ymax = y+ci)
plt.xlabel("Q functions")
plt.xticks(rotation=90)
plt.ylabel('E[||Q-Q*||_inf]')
plt.title("||Q-Q*||_inf")
plt.grid(visible=True)

In [8]:
for i, key in enumerate(intermediate_results.keys()):
    xmax = len(intermediate_results[key]['J_main_p'])*status_step
    x = np.linspace(0, xmax, num=len(intermediate_results[key]['J_main_p']))
    y = intermediate_results[key]['J_main_p']
    z = intermediate_results[key]['J_curr_p']
    plt.figure(figsize=(9, 4))
    #plt.scatter(x, y, c='orange', label='Current J')
    plt.hlines(y=J_star, xmin=0, xmax=xmax, colors=['green'], label='Optimal J')
    if "tau_" in key:
        plt.plot(x, y, label='J on main problem')
        plt.plot(x, z, label='J on '+str(key), c='orange')
    else:
         plt.plot(x, y, label='J on '+str(key))
    plt.grid(visible=True)
    plt.legend(loc="lower right")
    plt.xlabel('#Iterations')
    plt.ylabel('Expected discounted sum of returns')
    plt.title('Learning curves for '+str(key))

In [9]:
for i, key in enumerate(intermediate_results.keys()):
    xmax = len(intermediate_results[key]['delta_q'])*status_step
    x = np.linspace(0, xmax, num=len(intermediate_results[key]['delta_q']))
    y = intermediate_results[key]['delta_q']
    plt.figure(figsize=(9, 4))
    #plt.scatter(x, y, c='orange', label='Current DeltaQ')
    plt.plot(x, y, label='Current DeltaQ')
    plt.legend(loc="upper right")
    plt.xlabel('#Iterations')
    plt.ylabel('||Q-Q*||_inf')
    plt.grid(visible=True)
    plt.title('Learning curves for '+str(key))

In [10]:
for i, key in enumerate(intermediate_results.keys()):
    if "tau_" in key:
        xmax = len(intermediate_results[key]['delta_J'])*status_step
        x = np.linspace(0, xmax, num=len(intermediate_results[key]['delta_J']))
        y = np.array(intermediate_results[key]['J_main_p']) - np.array(intermediate_results[key]['J_curr_p'])
        z = intermediate_results[key]['l_bounds']
        plt.figure(figsize=(9, 4))
        #plt.scatter(x, y, c='orange', label='Current J_0 -J')
        plt.plot(x, y, label='Current J_0 -J')

        #plt.scatter(x, z, c='blue', label='Lower bound on J_0 -J')
        plt.plot(x, z, c='orange', label='Lower bound on J_0 - J')

        plt.grid(visible=True)
        plt.legend(loc="lower right")
        plt.xlabel('#Iterations')
        plt.ylabel('Performance difference')
        plt.title('Learning curves for '+str(key))

In [11]:
print(get_expected_avg_reward(tmdp.P_mat, get_policy(Q_star), tmdp.reward, tmdp.gamma, tmdp.mu))

365.78861003317843


In [12]:

def compute_tau_prime(A_hat, gamma, tau, Delta_Q):
    return (4*gamma**2*tau*Delta_Q - A_hat*(1-gamma))/(4*gamma**2*Delta_Q)


In [13]:
i = 0
for key in models.keys():
    if i == 0:
        tmdp = models[key]['tmdp']
        Q = models[key]['Q']
        d = compute_d(tmdp.mu, tmdp.P_mat_tau, get_policy(Q), tmdp.gamma)
        # Compute the gamma discounted state distribution
        delta = compute_delta(d, get_policy(Q))
        # Compute the state value function
        V = get_value_function(Q) 
        # Compute the expected reward when picking action a in state s
        r_s_a = compute_r_s_a(size, nA, tmdp.P_mat_tau, tmdp.reward)
        # Compute the state action next-state value function U_tau(s,a,s') = R(s,a) + \gamma*V_tau(s')
        U = compute_state_action_nextstate_value_function(size, nA, r_s_a, tmdp.gamma, V)
        # Rebuild Q using U as Q_tau(s,a) = \sum{s' \in S}P_tau(s'|s,a)*U_tau(s,a,s')
        Q_t = rebuild_Q_from_U(tmdp.P_mat_tau, U)



        # Compute the relative model advantage function hat \hat{A}_{tau, mu}(s,a)
        A_tau_hat = compute_relative_model_advantage_function_hat(tmdp.P_mat, tmdp.xi, U)
        # Compute the discounted distribution relative model advantage function hat \hat{A}_{tau, mu}
        A_hat = compute_discounted_distribution_relative_model_advantage_function_hat(A_tau_hat, delta)
        # The dissimilarity term D = D_e * gamma * D_inf is upperbounded by 4*gamma+(tau - tau_1)
        # Compute Delta Q_tau as the superior among the difference of the L_1 norm of elements of Q_tau
        d_q_t = get_sup_difference_q(Q_t)
        
        # Compute the performance improvement lower bound when moving to tau=0
        l_b = compute_performance_improvement_lower_bound(A_hat, tmdp.gamma, d_q_t, tmdp.tau, 0)

        tau_1 = compute_tau_prime(A_hat, tmdp.gamma, tmdp.tau, d_q_t)
        print("Learnt state action value function:\n",Q)
        print("Rebuilt state action value function\n",Q_t)
        print("Difference among Q_t and Q\n",Q_t - Q)
        print("L_inf norm of the difference: ", np.linalg.norm(Q_t-Q, np.inf))
        



Learnt state action value function:
 [[172.9714 174.942 ]
 [174.6771 165.4309]
 [173.0775 161.498 ]
 [170.1158 164.7669]
 [171.877  204.5195]
 [194.8566 245.2078]
 [219.9695 357.3799]
 [316.2082 669.6335]]
Rebuilt state action value function
 [[178.8982 174.716 ]
 [174.7732 174.256 ]
 [174.5825 172.9062]
 [173.4308 178.9427]
 [171.2983 202.3806]
 [196.069  246.6642]
 [225.3646 365.4989]
 [306.1285 638.5753]]
Difference among Q_t and Q
 [[  5.9267  -0.226 ]
 [  0.0961   8.8251]
 [  1.5049  11.4082]
 [  3.315   14.1758]
 [ -0.5788  -2.1389]
 [  1.2124   1.4564]
 [  5.3951   8.1189]
 [-10.0797 -31.0582]]
L_inf norm of the difference:  41.13794676623411
Learnt state action value function:
 [[ 74.1186  68.2915]
 [ 67.907   93.5923]
 [ 93.764  128.2993]
 [128.005  207.182 ]
 [170.0247 276.0783]
 [238.7521 401.5146]
 [348.3874 580.9712]
 [507.2324 794.5983]]
Rebuilt state action value function
 [[ 76.0369  76.2771]
 [ 71.1244  95.7661]
 [ 88.3002 136.723 ]
 [118.9118 199.7588]
 [188.4863 276.

In [14]:
done = False
intermediate_results = {}
curr_j = []
delta_qs = []
status_step = 200
models = {}
tau = 0.4
tmdp = TMDP(river, xi, tau, gamma)
m = 0
Q = np.zeros((tmdp.size, tmdp.nA))
# Learning from scratch with different values of tau
while not done:
    M = 30000
    
    s = tmdp.reset()
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    
    Q, inner_main_J, inner_curr_J, inner_delta_qs, inner_delta_J, inner_l_bounds = Q_learning(tmdp, s, a, Q, Q_star, M=M, alpha=0.03, debug=True, status_step=status_step, main_p=False)
    
    intermediate_results['tau_'+str(tau)] = {'J_main_p': inner_main_J,"J_curr_p":inner_curr_J, "delta_q":inner_delta_qs, 'Q':Q, "tmdp":tmdp, "delta_J":inner_delta_J, "l_bounds": inner_l_bounds}
    delta_q = np.linalg.norm(Q_star - Q, np.inf)
    
    J = get_expected_avg_reward(tmdp.P_mat, get_policy(Q), tmdp.reward, tmdp.gamma, tmdp.mu)
    results.append({"type":"scratch", "Q": Q, "tau":"{:.2f}".format(tau), "J":J, "policy":get_policy(Q),"delta_q":delta_q, "tmdp":tmdp})
    models[str(tau)] = {'Q':Q, 'tmdp':tmdp}
    
    d = compute_d(tmdp.mu, tmdp.P_mat_tau, get_policy(Q), tmdp.gamma)
    # Compute the gamma discounted state distribution
    delta = compute_delta(d, get_policy(Q))
    # Compute the state value function
    V = get_value_function(Q) 
    # Compute the expected reward when picking action a in state s
    r_s_a = compute_r_s_a(size, nA, tmdp.P_mat_tau, tmdp.reward)
    # Compute the state action next-state value function U_tau(s,a,s') = R(s,a) + \gamma*V_tau(s')
    U = compute_state_action_nextstate_value_function(size, nA, r_s_a, tmdp.gamma, V)
    # Rebuild Q using U as Q_tau(s,a) = \sum{s' \in S}P_tau(s'|s,a)*U_tau(s,a,s')
    Q_t = rebuild_Q_from_U(tmdp.P_mat_tau, U)

    # Compute the relative model advantage function hat \hat{A}_{tau, mu}(s,a)
    A_tau_hat = compute_relative_model_advantage_function_hat(tmdp.P_mat, tmdp.xi, U)
    # Compute the discounted distribution relative model advantage function hat \hat{A}_{tau, mu}
    A_hat = compute_discounted_distribution_relative_model_advantage_function_hat(A_tau_hat, delta)
    # The dissimilarity term D = D_e * gamma * D_inf is upperbounded by 4*gamma+(tau - tau_1)
    # Compute Delta Q_tau as the superior among the difference of the L_1 norm of elements of Q_tau
    d_q_t = get_sup_difference_q(Q_t)
    
    # Compute the performance improvement lower bound when moving to tau=0
    l_b = compute_performance_improvement_lower_bound(A_hat, tmdp.gamma, d_q_t, tmdp.tau, 0)

    tau = compute_tau_prime(A_hat, tmdp.gamma, tmdp.tau, d_q_t)
    print("Moving from {} to {}".format(tmdp.tau, tau))
    tmdp = TMDP(river, xi, tau, gamma)
    m+=1
    if m==20:
        done=True
    

Moving from 0.4 to 0.40054436831930434
Moving from 0.40054436831930434 to 0.40127300642644564
Moving from 0.40127300642644564 to 0.40237105462918515
Moving from 0.40237105462918515 to 0.40300280457334514
Moving from 0.40300280457334514 to 0.40346844305592905
Moving from 0.40346844305592905 to 0.4044097668988554
Moving from 0.4044097668988554 to 0.4052641651800538
Moving from 0.4052641651800538 to 0.40587531139255445
Moving from 0.40587531139255445 to 0.40654741303546155
Moving from 0.40654741303546155 to 0.4072025928778129
Moving from 0.4072025928778129 to 0.4076239483856288
Moving from 0.4076239483856288 to 0.4081921103279342
Moving from 0.4081921103279342 to 0.40896640625550523
Moving from 0.40896640625550523 to 0.40937189901430776
Moving from 0.40937189901430776 to 0.4095753141191535
Moving from 0.4095753141191535 to 0.4100404525341805
Moving from 0.4100404525341805 to 0.41076375234293094
Moving from 0.41076375234293094 to 0.4114298407921722
Moving from 0.4114298407921722 to 0.41228

In [15]:
for i, key in enumerate(intermediate_results.keys()):
    xmax = len(intermediate_results[key]['J_main_p'])*status_step
    x = np.linspace(0, xmax, num=len(intermediate_results[key]['J_main_p']))
    y = intermediate_results[key]['J_main_p']
    z = intermediate_results[key]['J_curr_p']
    plt.figure(figsize=(9, 4))
    #plt.scatter(x, y, c='orange', label='Current J')
    plt.hlines(y=J_star, xmin=0, xmax=xmax, colors=['green'], label='Optimal J')
    if "tau_" in key:
        plt.plot(x, y, label='J on main problem')
        plt.plot(x, z, label='J on '+str(key), c='orange')
    else:
         plt.plot(x, y, label='J on '+str(key))
    plt.grid(visible=True)
    plt.legend(loc="lower right")
    plt.xlabel('#Iterations')
    plt.ylabel('Expected discounted sum of returns')
    plt.title('Learning curves for '+str(key))

In [16]:
print(intermediate_results)

{'tau_0.4': {'J_main_p': [35.595799375000006, 13.528170161906171, 70.11234332645927, 75.34996502559424, 197.21930593693423, 197.21930593693423, 191.54866190494732, 165.20820760410382, 191.54866190494732, 165.20820760410382, 165.20820760410382, 165.20820760410382, 179.5844097601977, 189.0920662828275, 180.53384628694684, 180.53384628694684, 180.53384628694684, 180.53384628694684, 173.78424407763885, 180.53384628694684, 261.1985880066813, 179.5844097601977, 262.48415389883303, 273.8943101200351, 273.8943101200351, 316.06060621667064, 340.6844168677573, 340.6844168677573, 340.6844168677573, 293.05371165085023, 191.54866190494732, 316.06060621667064, 316.06060621667064, 316.06060621667064, 316.06060621667064, 316.06060621667064, 307.02578287933886, 307.02578287933886, 307.02578287933886, 307.02578287933886, 307.02578287933886, 307.02578287933886, 307.02578287933886, 316.06060621667064, 307.02578287933886, 302.3940379969599, 310.05861698072135, 310.05861698072135, 310.05861698072135, 310.05

In [17]:
V_star = get_value_function(Q_star)
U_star = compute_state_action_nextstate_value_function(size, nA, r_s_a, tmdp.gamma, V_star)

Q_t_Star = rebuild_Q_from_U(tmdp.P_mat, U_star)
print(Q_star - Q_t_Star)

[[ 1.8037e+00 -4.1061e-05]
 [ 0.0000e+00 -3.3170e-05]
 [ 0.0000e+00 -3.0129e-05]
 [ 0.0000e+00 -2.7447e-05]
 [ 0.0000e+00 -2.5029e-05]
 [ 0.0000e+00 -2.2832e-05]
 [ 0.0000e+00 -2.0831e-05]
 [ 0.0000e+00  7.2150e+01]]
